In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role

# 1. 配置基础信息
role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名
git_repo = 'YOUR_GITHUB_REPO_URL' # 替换为您的 GitHub 地址

# 定义 Git 配置
git_config = {'repo': git_repo, 'branch': 'main'}

# 定义训练数据路径
train_input = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/raw_data/', content_type='text/csv'
)

# ==========================================
# 定义三个 Estimator (不带超参数，超参数交给 Tuner)
# ==========================================

# 通用配置
common_args = {
    'entry_point': 'src/train.py',
    'source_dir': 'src',
    'role': role,
    'instance_count': 1,
    'instance_type': 'ml.m5.large', # 或 ml.c5.xlarge
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    'dependencies': ['requirements.txt'] # 自动安装 xgboost 等依赖
}

# 1. Random Forest Estimator
rf_estimator = SKLearn(**common_args)

# 2. SVM Estimator
svm_estimator = SKLearn(**common_args)

# 3. XGBoost Estimator
xgb_estimator = SKLearn(**common_args)

# ==========================================
# 定义超参数搜索空间 (Tuning Jobs)
# ==========================================

# 1. Random Forest Tuning
rf_hyperparameter_ranges = {
    'n_estimators': IntegerParameter(50, 200),
    'max_depth': IntegerParameter(5, 20),
}
# 固定 model_type 参数
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy', # 这里的名称要匹配脚本里打印的日志格式，或者自定义metric definition
    # 简便起见，SageMaker SKLearn 容器不一定能直接解析 print 的日志，
    # 我们可以通过 regex 告诉它怎么找 Accuracy。
    metric_definitions=[{'Name': 'accuracy', 'Regex': 'Accuracy: ([0-9\\.]+)'}],
    hyperparameter_ranges=rf_hyperparameter_ranges,
    max_jobs=4,    # 总共跑 4 次尝试
    max_parallel_jobs=2, # 每次并行跑 2 个
    base_tuning_job_name='rf-tuning'
)

# 2. SVM Tuning
svm_hyperparameter_ranges = {
    'C': ContinuousParameter(0.1, 10.0),
    'kernel': CategoricalParameter(['rbf', 'linear', 'poly'])
}
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=[{'Name': 'accuracy', 'Regex': 'Accuracy: ([0-9\\.]+)'}],
    hyperparameter_ranges=svm_hyperparameter_ranges,
    max_jobs=4,
    max_parallel_jobs=2,
    base_tuning_job_name='svm-tuning'
)

# 3. XGBoost Tuning
xgb_hyperparameter_ranges = {
    'n_estimators_xgb': IntegerParameter(50, 200),
    'learning_rate': ContinuousParameter(0.01, 0.3)
}
xgb_estimator.set_hyperparameters(model_type='xgboost')

xgb_tuner = HyperparameterTuner(
    estimator=xgb_estimator,
    objective_metric_name='accuracy',
    metric_definitions=[{'Name': 'accuracy', 'Regex': 'Accuracy: ([0-9\\.]+)'}],
    hyperparameter_ranges=xgb_hyperparameter_ranges,
    max_jobs=4,
    max_parallel_jobs=2,
    base_tuning_job_name='xgb-tuning'
)

# ==========================================
# 发射！(Launch Jobs)
# ==========================================
print("🚀 Launching Random Forest Tuning Job...")
rf_tuner.fit({'train': train_input}, wait=False) # wait=False 让它后台跑，我们可以继续发下一个

print("🚀 Launching SVM Tuning Job...")
svm_tuner.fit({'train': train_input}, wait=False)

print("🚀 Launching XGBoost Tuning Job...")
xgb_tuner.fit({'train': train_input}, wait=True) # 最后一个 wait=True，等待结束

print("✅ All tuning jobs launched!")

In [ ]:
# 获取最佳结果
rf_best_name = rf_tuner.best_training_job()
svm_best_name = svm_tuner.best_training_job()
xgb_best_name = xgb_tuner.best_training_job()

print(f"🏆 Best RF Job: {rf_best_name}")
print(f"🏆 Best SVM Job: {svm_best_name}")
print(f"🏆 Best XGB Job: {xgb_best_name}")

# 您可以调用 describe 来查看具体的 Accuracy 分数
# 也可以直接去 SageMaker 控制台或 W&B 看图表
# 假设 XGBoost 最好：
best_model_s3_uri = sagemaker_session.describe_training_job(xgb_best_name)['ModelArtifacts']['S3ModelArtifacts']
print(f"\n🎉 最终选定的模型 S3 路径: {best_model_s3_uri}")
print("请将此路径用于后续的 Docker 部署！")